# Generated Geometry with Architecture 16R

Through her thesis "3D Generative Adversarial Networks to Autonomously Generate Building Geometry", Lisa-Marie Mueller researched how Generative Adversarial Networks can be used to produce building geometry. Through the exploration completed in the thesis, two architectures performed the best. Architecture 16R uses Leaky ReLU in the generator and the critic, uses RMSProp as the optimizer with a fixed learning rate, and implements gradient penalty. The architectures has 10 layers with the following number of channels 96-96-48-48-24-24-12-12-2-2.

This notebook loads the weights of the trained network and allows users to generate new geometry using the trained network models. The generated geometry is then visualized in the notebook. 

Before running this notebook, please make sure tensorflow is installed in your conda environment and that you activated this environment. An env file is included on GitHub.

In [ ]:
#imports and initialize variabless
import tensorflow as tf
import os
import PIL
from PIL import Image
import math

import wganv16R as gan
import utilities.ganutilities as util

save_location = 'generated/images'
generated_matrices = 'generated/generated_matrices'

In [ ]:
#load network weights
generator = gan.make_generator_model()
discriminator = gan.make_discriminator_model()

generator_optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.00005)
discriminator_optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.00005)

checkpoint_dir = 'sample_files/training_checkpoints/16R/ckpt-112'
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)

manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep = 3)

checkpoint.restore(checkpoint_dir)

In [ ]:
#thank you for your patience while this cell runs
#optional: change how many samples to generate
number_of_samples_to_generate = 4

#generate geometry with trained network
if number_of_samples_to_generate < 11:
    seed = tf.random.normal([number_of_samples_to_generate, 200])
    util.save_generated_matrix('16Rsamples', generator, 0, seed)
    print("SUCCESSFUL: Generated Samples")
else:
    print("FAILED: value of number_of_samples_to_generate must be less than or equal to 10")

In [ ]:
#thank you for your patience while this cell runs
#create and save images of generated matrices
matrices_location = generated_matrices + '/16Rsamples'
util.visualize_files_from_folder(matrices_location, save_location, '16Rsamples')

In [ ]:
#show saved images of geometry
images = []
image_path = save_location + '/16Rsamples'
all_files = [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]

images = [Image.open(image_path + '/' + x) for x in all_files]

new_im = Image.new('RGB', (1000, 330 * (1 + math.ceil(len(all_files)/3))), (255, 255, 255))

for i, im in enumerate(images):
    width, height = im.size
    target_width = 330 #px
    ratio = width / target_width
    new_height = int(height // ratio)

    im_resized = im.resize((target_width, new_height))
    
    new_im.paste(im_resized, (im_resized.size[0] * (i - (math.floor(i / 3) * 3)), math.floor(i / 3) * im_resized.size[1]))

display(new_im)